In [1]:
import sys

sys.path.append('/home/max/myrepos/vlstm_cuda/')

import torch
torch.set_printoptions(linewidth=200, threshold=100000)
import triton
import triton.language as tl
import math

In [2]:
triton.runtime.driver.active.get_current_target()

GPUTarget(backend='cuda', arch=89, warp_size=32)

In [3]:
from src_triton.mlstm_parallel_fw_v0.qkv_causal import qkv_fw
from src_triton.utils import lower_triangular_block_matrix

In [4]:
list(range(0,0*8,4))

[]

In [5]:
S = 128 #32 #32 #16 #8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 32 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [6]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(1) #TODO from here: with seed=0 even the pytorch version alone breaks for float16 and bfloat16
# fixed:
# qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
# qs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# # vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# vs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 100.
# # vs = torch.zeros((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# vs[:,:,1,0] = 7.
# qs[:,:,1,0] = 1.

# vs[:,:,1,16] = 8.
# qs[:,:,1,16] = 1.
# random: 
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1) / 10.
# igs = torch.zeros((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
igs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
# fgs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
fgs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE)

dHs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)

In [7]:
# inputs float16
dtype_fp16 = torch.float16
qs_half = qs.to(dtype=dtype_fp16)
ks_half = ks.to(dtype=dtype_fp16)
vs_half = vs.to(dtype=dtype_fp16)
igs_half = igs.to(dtype=dtype_fp16)
fgs_half = fgs.to(dtype=dtype_fp16)
dHs_half = dHs.to(dtype=dtype_fp16)

In [8]:
# inputs bfloat16
dtype_bf16 = torch.bfloat16
qs_bf16 = qs.to(dtype=dtype_bf16)
ks_bf16 = ks.to(dtype=dtype_bf16)
vs_bf16 = vs.to(dtype=dtype_bf16)
igs_bf16 = igs.to(dtype=dtype_bf16)
fgs_bf16 = fgs.to(dtype=dtype_bf16)
dHs_bf16 = dHs.to(dtype=dtype_bf16)

In [9]:
hs_half, m_half, n_half = qkv_fw(matQ=qs_half, matK=ks_half, matV=vs_half)

Triton grid: (2, 1, 1), BLOCK_Q: 64, BLOCK_KV: 32


loc("/home/max/myrepos/vlstm_cuda/src_triton/mlstm_parallel_fw_v0/qkv_causal.py":142:11): error: operation scheduled before its operands


In [10]:
ltr_bm = lower_triangular_block_matrix(S, 64, DEVICE, dtype=dtype_fp16)

In [21]:
qkv_ltr_bm_half = (((qs_half @ ks_half.transpose(-2, -1))/math.sqrt(DH)) * ltr_bm) @ vs_half
qkv_ltr_half = (((qs_half @ ks_half.transpose(-2, -1))/math.sqrt(DH)).tril()) @ vs_half

In [12]:
hs_half[:, :, :10, :10]

tensor([[[[ 0.0556,  0.0978, -0.0798, -0.0310, -0.0456, -0.0714,  0.0558, -0.1772, -0.0195, -0.0232],
          [ 0.4043, -0.4331,  2.3691, -0.4629, -0.8281,  0.6704, -1.7412,  0.5815, -0.9185,  1.2676],
          [-0.3669,  0.0252, -1.2998,  2.0898,  2.5176, -0.6919,  0.1941, -0.1479,  1.1895, -0.7510],
          [-1.5811, -3.2598, -1.6387, -0.6538,  0.8394,  2.1777,  1.9600,  5.5820,  1.7295, -0.9033],
          [ 2.1289,  4.9453,  3.5410, -1.0029, -1.8477, -0.9902, -2.0371, -5.7070, -0.3657,  1.8633],
          [-0.6030, -0.3259, -0.2749, -1.3096,  1.0742,  3.0039,  1.0791,  2.6875,  1.7773,  0.4509],
          [ 0.0750, -4.1094, -0.3271,  0.9980,  1.9805, -0.9873, -0.7314,  5.1094,  2.4902, -1.1396],
          [-3.4785, -8.0625, -3.8281, -4.5938, -3.9336, -2.2715,  1.2090,  1.7812, -4.9805, -4.3516],
          [-2.4160,  3.4512, -3.9473,  2.7832,  4.2344, -1.8193,  1.7090,  1.1094,  1.1631, -4.5156],
          [ 0.6387, -5.1914,  2.9902, -2.9297,  0.0646,  2.2695, -1.5391,  0.6274,

In [22]:
qkv_ltr_half[:, :, :10, :10]

tensor([[[[ 0.0556,  0.0978, -0.0797, -0.0310, -0.0456, -0.0714,  0.0558, -0.1771, -0.0195, -0.0231],
          [ 0.4043, -0.4331,  2.3691, -0.4629, -0.8281,  0.6704, -1.7412,  0.5815, -0.9185,  1.2676],
          [-0.3669,  0.0252, -1.2998,  2.0898,  2.5176, -0.6919,  0.1941, -0.1479,  1.1895, -0.7510],
          [-1.5811, -3.2598, -1.6367, -0.6533,  0.8394,  2.1777,  1.9590,  5.5859,  1.7295, -0.9028],
          [ 2.1289,  4.9453,  3.5410, -1.0029, -1.8477, -0.9902, -2.0371, -5.7070, -0.3657,  1.8633],
          [-0.6030, -0.3262, -0.2751, -1.3096,  1.0752,  3.0039,  1.0791,  2.6875,  1.7773,  0.4509],
          [ 0.0748, -4.1094, -0.3269,  0.9980,  1.9805, -0.9868, -0.7319,  5.1094,  2.4902, -1.1396],
          [-3.4785, -8.0625, -3.8281, -4.5938, -3.9336, -2.2715,  1.2100,  1.7822, -4.9805, -4.3516],
          [-2.4160,  3.4512, -3.9453,  2.7812,  4.2344, -1.8193,  1.7070,  1.1094,  1.1621, -4.5156],
          [ 0.6382, -5.1875,  2.9922, -2.9297,  0.0643,  2.2695, -1.5391,  0.6270,

In [23]:
res_diff = hs_half - qkv_ltr_half

In [25]:
res_diff[:, :, :10, :10], res_diff.shape

(tensor([[[[ 3.0518e-05,  6.1035e-05, -6.1035e-05, -1.5259e-05, -3.0518e-05, -6.1035e-05,  3.0518e-05, -1.2207e-04, -1.5259e-05, -1.5259e-05],
           [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
           [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
           [ 0.0000e+00,  0.0000e+00, -1.9531e-03, -4.8828e-04,  0.0000e+00,  0.0000e+00,  9.7656e-04, -3.9062e-03,  0.0000e+00, -4.8828e-04],
           [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
           [ 0.0000e+00,  2.4414e-04,  2.4414e-04,  0.0000e+00, -9.7656e-04,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
           [ 1.2207e-04,  0.0000e+00, -2.4414e-04,  0.0000e+00,  0.0000e+00, -4.8828e-04,  4.8828e-04,  0.0000e+00,  0.0000e+00,  0.0000e+00],

In [16]:
torch.max(torch.abs(res_diff))

tensor(30.5000, device='cuda:0', dtype=torch.float16)

In [17]:
qs_half.stride(0), qs_half.stride(1), qs_half.stride(2), qs_half.stride(3)

(4096, 4096, 32, 1)

In [18]:
3 & 1

1

In [19]:
bool(3 & 2)

True